# Classification and regression models

In [1]:
import rebound
from spock import FeatureClassifier
feature_model = FeatureClassifier()

sim = rebound.Simulation()
sim.add(m=1.)
sim.add(m=1.e-5, P=1., e=0.03, pomega=2., l=0.5)
sim.add(m=1.e-5, P=1.2, e=0.03, pomega=3., l=3.)
sim.add(m=1.e-5, P=1.5, e=0.03, pomega=1.5, l=2.)
sim.move_to_com()

print(feature_model.predict_stable(sim))

/Users/dtamayo/opt/miniconda3/envs/p3/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [09:58:07] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:873: Found JSON model saved before XGBoost 1.6, please save the model using current version again. The support for old JSON model will be discontinued in XGBoost 2.3.
  warnings.warn(smsg, UserWarning)


AttributeError: 'Simulation' object has no attribute 'calculate_megno'

In [ ]:
from spock import DeepRegressor
deep_model = DeepRegressor()

median, lower, upper = deep_model.predict_instability_time(sim, samples=10000)
print(int(median))

In [ ]:
from spock import class_MLP, CollisionClassifier #for some reason, you need to import class_MLP here
class_model = CollisionClassifier()

prob_12, prob_23, prob_13 = class_model.predict_collision_probs(sim)
print(prob_12, prob_23, prob_13)

In [ ]:
from spock import reg_MLP, CollisionRegressor #for some reason, you need to import reg_MLP here
reg_model = CollisionRegressor()

a1, a2, e1, e2, inc1, inc2 = reg_model.predict_collision_outcome(sim, collision_inds=[2, 3]) #inclinations are measured wrt to L axis (coplanar is maybe a bad example...)
print(a1, a2, e1, e2, inc1, inc2)

# Iterative planet formation predictions

In [ ]:
sim = rebound.Simulation()
sim.add(m=1.)
sim.add(m=1.e-5, P=1., e=0.03, pomega=2., l=0.5)
sim.add(m=1.e-5, P=1.2, e=0.03, pomega=3., l=3.)
sim.add(m=1.e-5, P=1.5, e=0.03, pomega=1.5, l=2.)
sim.add(m=1.e-5, P=2.2, e=0.03, pomega=0.5, l=5.0)
sim.add(m=1.e-5, P=2.5, e=0.03, pomega=5.0, l=1.5)
sim.move_to_com()

In [ ]:
from spock import iterative_model
PF_map = iterative_model.PlanetFormationMap()

In [ ]:
final_state = PF_map.predict_final_system(sim)
final_state

In [ ]:
ps = final_state.particles
for i in range(1, len(ps)):
    print('m' + str(i) + ':', ps[i].m)
    print('a' + str(i) + ':', ps[i].a)
    print('e' + str(i) + ':', ps[i].e)
    print('inc' + str(i) + ':', ps[i].inc, '\n')